## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/luca/teaching/simplon/IA_2024/teaching-resources/Ressources/MLOps/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1730210315440, experiment_id='1', last_update_time=1730210315440, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/luca/teaching/simplon/IA_2024/teaching-resources/Ressources/MLOps/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1730210315418, experiment_id='0', last_update_time=1730210315418, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f0c472b508b548888b18f85b5616940b, rmse: 6.3184
run id: f74aba41f53c45afb5062f154c472f11, rmse: 6.3534
run id: a699c32be21945d1ae436ca0cf10909e, rmse: 6.3844
run id: 25f1ea6c08e942d4a61a86128a60088b, rmse: 6.4111
run id: 241474ddbdb64e488c3550b858b3706e, rmse: 6.4649


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.
4. Or the new way, see below.

#### 1. Register a new version for the experiment `nyc-taxi-regressor`

In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [38]:
run_id = "9bdaa59d69124393b3804bb516872482"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="staging.nyc-taxi-regressor")

Registered model 'staging.nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'staging.nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1730648653018, current_stage='None', description=None, last_updated_timestamp=1730648653018, name='staging.nyc-taxi-regressor', run_id='9bdaa59d69124393b3804bb516872482', run_link=None, source='/home/luca/teaching/simplon/IA_2024/teaching-resources/Ressources/MLOps/mlops-zoomcamp/02-experiment-tracking/mlruns/1/9bdaa59d69124393b3804bb516872482/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

#### 2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version was created.

In [ ]:
# old way
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_7275/3547440689.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [40]:
# new way
model_name = "staging.nyc-taxi-regressor"
versions = client.search_model_versions(filter_string =f"name='{model_name}'", order_by=["version_number DESC"])
# uncomment the following line to see the entire output
# versions_dict = list(map(lambda x: dict(x), versions))
# display(versions_dict)

for version in versions:
    print(f"version: {version.version}, aliases: {version.aliases}")

latest_version = versions[0]

version: 4, aliases: []
version: 3, aliases: []
version: 1, aliases: []


#### 3. Transition the latest version to "Staging" and adding annotations to it.

In [ ]:
# old way
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972141519, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

#### 4. Copy source model version into a new registered model

In [42]:
# new way
# Copy the source model version into a new registered model
dst_name = "production.nyc-taxi-regressor"
src_model_uri = f"models:/{latest_version.name}/{latest_version.version}"
mv_copy = client.copy_model_version(src_model_uri, dst_name)

print(f"Name: {mv_copy.name}")
print(f"Version: {mv_copy.version}")
print(f"Source: {mv_copy.source}")

Name: production.nyc-taxi-regressor
Version: 2
Source: models:/staging.nyc-taxi-regressor/4


Registered model 'production.nyc-taxi-regressor' already exists. Creating a new version of this model...
Copied version '4' of model 'staging.nyc-taxi-regressor' to version '2' of model 'production.nyc-taxi-regressor'.


In [ ]:
client.set_registered_model_alias(name=dst_name, alias="champion", version=mv_copy.version)

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [35]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

# old way
def test_model_old(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

# new way
def test_model(name, version, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{version}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [36]:
df = read_dataframe("../data/green_tripdata_2021-03.csv")

/tmp/ipykernel_7275/3640186763.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [45]:
# path to a run with preprocessor artifact
run_id_with_artifacts = "8440c5c2f924488291ce4a0fe9451de1"
client.download_artifacts(run_id=run_id_with_artifacts, path='preprocessor', dst_path='.')

'/home/luca/teaching/simplon/IA_2024/teaching-resources/Ressources/MLOps/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [46]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [47]:
X_test = preprocess(df, dv)

In [48]:
target = "duration"
y_test = df[target].values

In [54]:
client.get_model_version_by_alias("production.nyc-taxi-regressor", "champion")


<ModelVersion: aliases=['champion'], creation_timestamp=1730648711795, current_stage='None', description=None, last_updated_timestamp=1730648711795, name='production.nyc-taxi-regressor', run_id='9bdaa59d69124393b3804bb516872482', run_link=None, source='models:/staging.nyc-taxi-regressor/4', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [55]:
# old way
# %time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

# new way
model_name='production.nyc-taxi-regressor'
model_version = client.get_model_version_by_alias(model_name, "champion")
%time test_model(name=model_name, version=model_version.version, X_test=X_test, y_test=y_test)

CPU times: user 2.35 s, sys: 42.8 ms, total: 2.39 s
Wall time: 852 ms


/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': np.float64(6.45868087743356)}

In [64]:
# old way
# %time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

# new way
model_name='staging.nyc-taxi-regressor'
# with latest version
# latest_model_version = client.get_latest_versions(model_name)[0]
# %time test_model(name=model_name, version=latest_model_version.version, X_test=X_test, y_test=y_test)

%time test_model(name=model_name, version=5, X_test=X_test, y_test=y_test)

CPU times: user 104 ms, sys: 283 μs, total: 104 ms
Wall time: 142 ms


/home/luca/.pyenv/versions/3.10.6/envs/mlops_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': np.float64(7.720735234972245)}

In [ ]:
# old way
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>